In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path

from sdm_eurec4a.visulization import set_custom_rcParams

set_custom_rcParams()

['#CC6677', '#6E9CB3', '#CA8727', '#44AA99', '#AA4499', '#D6BE49', '#A494F5']

In [ ]:
from pySD.sdmout_src import supersdata
from pySD.sdmout_src import pygbxsdat, pysetuptxt, supersdata

In [ ]:
data_dir = Path("/home/m/m301096/CLEO/data/output_v4.1/condensation/cluster_100/")

cloud_id = int(data_dir.name.split("_")[1])

output_dir = data_dir / "processed"
output_dir.mkdir(exist_ok=True, parents=False)

output_path = output_dir / "eulerian_dataset.nc"
output_path.parent.mkdir(exist_ok=True)

setupfile_path = data_dir / "config" / "eurec4a1d_setup.txt"
statsfile_path = data_dir / "config" / "eurec4a1d_stats.txt"
zarr_path = data_dir / "eurec4a1d_sol.zarr"
gridfile_path = data_dir / "share/eurec4a1d_ddimlessGBxboundaries.dat"

# read in constants and intial setup from setup .txt file
config = pysetuptxt.get_config(str(setupfile_path), nattrs=3, isprint=False)
consts = pysetuptxt.get_consts(str(setupfile_path), isprint=False)
gridbox_dict = pygbxsdat.get_gridboxes(str(gridfile_path), consts["COORD0"], isprint=False)

ds_zarr = xr.open_zarr(zarr_path, consolidated=False)
ds_zarr = ds_zarr.rename({"gbxindex": "gridbox"})
ds_zarr = ds_zarr.compute()

# Use the SupersDataNew class to read the dataset
dataset = supersdata.SupersDataNew(dataset=str(zarr_path), consts=consts)

dataset.set_attribute(dataset["sdId"].attribute_to_indexer_unique())
dataset.index_by_indexer(dataset["sdId"])

Reading binary file:
 /home/m/m301096/CLEO/data/output_v4.1/condensation/cluster_100/share/eurec4a1d_ddimlessGBxboundaries.dat
---- Superdrop Properties -----
RHO_L = 998.203 Kg/m^3
RHO_SOL = 2016.5 Kg/m^3
MR_SOL = 0.05844277 Kg/mol
IONIC = 2.0
-------------------------------
supers dataset path:  /home/m/m301096/CLEO/data/output_v4.1/condensation/cluster_100/eurec4a1d_sol.zarr
Attribute coord1 not found in dataset
Attribute coord2 not found in dataset


In [ ]:
number_above_zero = (ds_zarr["xi"] > 0).sum().values
number_equal_zero = (ds_zarr["xi"] == 0).sum().values
number_below_zero = (ds_zarr["xi"] < 0).sum().values

print(f"Number of xi above zero: {number_above_zero}")
print(f"Number of xi equal to zero: {number_equal_zero}")
print(f"Number of xi below zero: {number_below_zero}")

Number of xi above zero: 576253
Number of xi equal to zero: 484049
Number of xi below zero: 0


In [ ]:
import matplotlib.pyplot as plt
import awkward as ak

r = ak.flatten(dataset["radius"].data)
xi = ak.flatten(dataset["xi"].data)

mask = xi == 0

plt.plot(r[~mask], xi[~mask], ".", label=r"$\xi$ == 0")

plt.plot(r[mask], xi[mask], ".", label=r"$\xi$ != 0")
plt.xscale("log")
plt.legend()
# plt.ylim(0, 4)
plt.xlabel("Radius in µm")
plt.ylabel(r"Multiplicity $\xi$")
plt.title(rf"Cloud {cloud_id} Validate $\xi$")

Text(0.5, 1.0, 'Cloud 100 Validate $\\xi$')

In [ ]:
import matplotlib.pyplot as plt
import awkward as ak

r = dataset["radius"].data
xi = dataset["xi"].data

mask = ak.num(r) > 10

r[mask]
xi[mask]